<h2> 3.6 Featurizing text data with tfidf  </h2>

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [6]:
# avoid decoding problems
df = pd.read_csv("train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [7]:
df.head()
print(df['question1'].values)

['What is the step by step guide to invest in share market in india?'
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?'
 'How can I increase the speed of my internet connection while using a VPN?'
 ... 'What is one coin?'
 'What is the approx annual cost of living while studying in UIC Chicago, for an Indian student?'
 'What is like to have sex with cousin?']


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf_q1= TfidfVectorizer(lowercase=False,)

q1_tfidf=tfidf_q1.fit_transform(df['question1'].values.astype('U'))

tfidf_q2= TfidfVectorizer(lowercase=False,)
q2_tfidf=tfidf_q2.fit_transform(df['question2'].values.astype('U'))

# dict key:word and value:tf-idf score
#word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))




print("Found {0} features from question2 column".format(len(tfidf_q2.get_feature_names())))



Found 78351 features from question2 column


In [53]:
print(q1_tfidf.shape)
print(q2_tfidf.shape)
from scipy.sparse import hstack
question1_question2 = hstack((q1_tfidf,q2_tfidf)).tocsr()


print(question1_question2.shape)

print(type(question1_question2))


(404290, 84717)
(404290, 78351)
(404290, 163068)
<class 'scipy.sparse.csr.csr_matrix'>


- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [10]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [55]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

df3  = df1.merge(df2, on='id',how='left')
print(df3.shape)
   
df4 = hstack((df3,question1_question2)).tocsr()
print(df4.shape)

(404290, 28)
(404290, 163096)


In [26]:
# dataframe of nlp features
df1.head()

id  is_duplicate   cwc_min   cwc_max   csc_min   csc_max   ctc_min  \
0   0             0  0.999980  0.833319  0.999983  0.999983  0.916659   
1   1             0  0.799984  0.399996  0.749981  0.599988  0.699993   
2   2             0  0.399992  0.333328  0.399992  0.249997  0.399996   
3   3             0  0.000000  0.000000  0.000000  0.000000  0.000000   
4   4             0  0.399992  0.199998  0.999950  0.666644  0.571420   

    ctc_max  last_word_eq  first_word_eq  abs_len_diff  mean_len  \
0  0.785709           0.0            1.0           2.0      13.0   
1  0.466664           0.0            1.0           5.0      12.5   
2  0.285712           0.0            1.0           4.0      12.0   
3  0.000000           0.0            0.0           2.0      12.0   
4  0.307690           0.0            1.0           6.0      10.0   

   token_set_ratio  token_sort_ratio  fuzz_ratio  fuzz_partial_ratio  \
0              100                93          93                 100   
1               86                63          66                  75   
2               66                66          54                  54   
3               36                36          35                  40   
4               67                47          46                  56   

   longest_substr_ratio  
0              0.982759  
1              0.596154  
2              0.166667  
3              0.039216  
4              0.175000

In [27]:
# data before preprocessing 
df2.head()

id  freq_qid1  freq_qid2  q1len  q2len  q1_n_words  q2_n_words  \
0   0          1          1     66     57          14          12   
1   1          4          1     51     88           8          13   
2   2          1          1     73     59          14          10   
3   3          1          1     50     65          11           9   
4   4          3          1     76     39          13           7   

   word_Common  word_Total  word_share  freq_q1+q2  freq_q1-q2  
0         10.0        23.0    0.434783           2           0  
1          4.0        20.0    0.200000           5           3  
2          4.0        24.0    0.166667           2           0  
3          0.0        19.0    0.000000           2           0  
4          2.0        20.0    0.100000           4           2

In [28]:
df3.head()

id  is_duplicate   cwc_min   cwc_max   csc_min   csc_max   ctc_min  \
0   0             0  0.999980  0.833319  0.999983  0.999983  0.916659   
1   1             0  0.799984  0.399996  0.749981  0.599988  0.699993   
2   2             0  0.399992  0.333328  0.399992  0.249997  0.399996   
3   3             0  0.000000  0.000000  0.000000  0.000000  0.000000   
4   4             0  0.399992  0.199998  0.999950  0.666644  0.571420   

    ctc_max  last_word_eq  first_word_eq     ...      freq_qid2  q1len  q2len  \
0  0.785709           0.0            1.0     ...              1     66     57   
1  0.466664           0.0            1.0     ...              1     51     88   
2  0.285712           0.0            1.0     ...              1     73     59   
3  0.000000           0.0            0.0     ...              1     50     65   
4  0.307690           0.0            1.0     ...              1     76     39   

   q1_n_words  q2_n_words  word_Common  word_Total  word_share  freq_q1+q2  \
0          14          12         10.0        23.0    0.434783           2   
1           8          13          4.0        20.0    0.200000           5   
2          14          10          4.0        24.0    0.166667           2   
3          11           9          0.0        19.0    0.000000           2   
4          13           7          2.0        20.0    0.100000           4   

   freq_q1-q2  
0           0  
1           3  
2           0  
3           0  
4           2  

[5 rows x 28 columns]